# Test weighted average

In [1]:
import sys
sys.path.insert(0,"/home/gridsan/hwpang/Software/RMG-Py/")
sys.path.insert(0,"..")

import json

import random
import time as time
import pandas as pd
import numpy as np

from rmgpy.molecule.group import Group
from rmgpy.data.base import Entry
from rmgpy.data.thermo import ThermoData

from tree.thermo import ThermoGroups, average_thermo_data
from tree.utils import make_mol
from tree.parameters import Ts

# Get data

In [2]:
hbi_unc_df = pd.read_csv("../data/hbi_unc.csv")
hbi_unc_df

,radical_smiles,resonance_radical_smiles,resonance_radical_num_rotatable_bonds,radical_H298 (kcal/mol),radical_Sint298 (cal/mol/K),radical_source,radical_level_of_theory,radical_Cp300 (cal/mol/K),radical_Cp400 (cal/mol/K),radical_Cp500 (cal/mol/K),...,unc_closed_shell_Cp1500 (cal/mol/K),unc_HBI_H298 (kcal/mol),unc_HBI_Sint298 (cal/mol/K),unc_HBI_Cp300 (cal/mol/K),unc_HBI_Cp400 (cal/mol/K),unc_HBI_Cp500 (cal/mol/K),unc_HBI_Cp600 (cal/mol/K),unc_HBI_Cp800 (cal/mol/K),unc_HBI_Cp1000 (cal/mol/K),unc_HBI_Cp1500 (cal/mol/K)
0,[O]C(=O)OC(O)(O)O,[O]C(=O)OC(O)(O)O,4,-223.514126,95.999974,dong_pio_liang.py,CBS-QB3,33.926878,38.387673,41.508496,...,2.0,4.176123,2.236068,2.236068,2.236068,2.236068,2.236068,2.236068,2.236068,2.236068
1,[O]C(O)(O)OC(=O)O,[O]C(O)(O)OC(=O)O,3,-223.423390,92.781436,dong_pio_liang.py,CBS-QB3,32.531179,37.569659,41.498443,...,2.0,4.176123,2.236068,2.236068,2.236068,2.236068,2.236068,2.236068,2.236068,2.236068
2,O=C(O)O[C](O)O,O=C(O)O[C](O)O,3,-187.322869,89.517068,dong_pio_liang.py,CBS-QB3,28.973242,34.553073,38.746721,...,2.0,4.176123,2.236068,2.236068,2.236068,2.236068,2.236068,2.236068,2.236068,2.236068
3,CC(=O)OC(OO)C(=O)C(O)O[O],CC(=O)OC(OO)C(=O)C(O)O[O],8,-185.660577,132.633380,dong_pio_liang.py,CBS-QB3,56.569703,66.028226,73.603744,...,2.0,4.176123,2.236068,2.236068,2.236068,2.236068,2.236068,2.236068,2.236068,2.236068
4,O=[C]OC(O)(O)O,O=[C]OC(O)(O)O,5,-180.497563,90.031975,dong_pio_liang.py,CBS-QB3,32.378191,36.844343,39.334179,...,2.0,4.176123,2.236068,2.236068,2.236068,2.236068,2.236068,2.236068,2.236068,2.236068
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2800,C1=C[CH]C=1,[C]1=CC=C1,0,156.106582,65.385768,dong_pio_liang.py,CBS-QB3,14.653111,18.488757,21.562144,...,2.0,4.176123,2.236068,2.236068,2.236068,2.236068,2.236068,2.236068,2.236068,2.236068
2801,C=C=C1C=[C]C1,C=C=C1C=[C]C1,0,157.733085,77.587625,dong_pio_liang.py,CBS-QB3,22.675255,28.333884,32.976246,...,2.0,4.176123,2.236068,2.236068,2.236068,2.236068,2.236068,2.236068,2.236068,2.236068
2802,[CH2]C1=CC#CC1,[CH2]C1=CC#CC1,1,163.498344,75.963844,dong_pio_liang.py,CBS-QB3,22.606638,28.495118,33.391100,...,2.0,4.176123,2.236068,2.236068,2.236068,2.236068,2.236068,2.236068,2.236068,2.236068
2803,[CH2]C1=CC#CC1,C=C1[CH]C#CC1,0,163.498344,75.963844,dong_pio_liang.py,CBS-QB3,22.606638,28.495118,33.391100,...,2.0,4.176123,2.236068,2.236068,2.236068,2.236068,2.236068,2.236068,2.236068,2.236068


In [3]:
trunc_hbi_unc_df = hbi_unc_df.sample(3, random_state=0)

In [4]:
HBI_corrections = trunc_hbi_unc_df.apply(lambda x: ThermoData(
    H298=(x["HBI_H298 (kcal/mol)"], "kcal/mol", "+|-", x["unc_HBI_H298 (kcal/mol)"]),
    S298=(x["HBI_Sint298 (cal/mol/K)"], "cal/mol/K", "+|-", x["unc_HBI_Sint298 (cal/mol/K)"]),
    Cpdata=([x[f"HBI_Cp{T} (cal/mol/K)"] for T in Ts], "cal/mol/K", "+|-", [x[f"unc_HBI_Cp{T} (cal/mol/K)"] for T in Ts]),
    Tdata=(Ts, "K"),
    comment=f"Radical thermo from {x['radical_source']} and closed shell thermo from {x['closed_shell_thermo_source'].replace('Thermo library: ../data/', '')}"
), axis=1)
HBI_corrections = HBI_corrections.to_list()

In [5]:
HBI_corrections = [HBI_corrections[0], HBI_corrections[0], HBI_corrections[0]]
HBI_corrections

[ThermoData(Tdata=([300,400,500,600,800,1000,1500],'K'), Cpdata=([-0.977157,-0.655546,-0.881567,-1.34064,-2.12884,-2.7591,-3.79178],'cal/mol/K','+|-',[1.41421,1.41421,1.41421,1.41421,1.41421,1.41421,1.41421]), H298=(81.6781,'kcal/mol','+|-',1.69706), S298=(-0.103705,'cal/mol/K','+|-',1.41421), comment="""Radical thermo from dong_pio_liang.py and closed shell thermo from dong_pio_liang.py"""),
 ThermoData(Tdata=([300,400,500,600,800,1000,1500],'K'), Cpdata=([-0.977157,-0.655546,-0.881567,-1.34064,-2.12884,-2.7591,-3.79178],'cal/mol/K','+|-',[1.41421,1.41421,1.41421,1.41421,1.41421,1.41421,1.41421]), H298=(81.6781,'kcal/mol','+|-',1.69706), S298=(-0.103705,'cal/mol/K','+|-',1.41421), comment="""Radical thermo from dong_pio_liang.py and closed shell thermo from dong_pio_liang.py"""),
 ThermoData(Tdata=([300,400,500,600,800,1000,1500],'K'), Cpdata=([-0.977157,-0.655546,-0.881567,-1.34064,-2.12884,-2.7591,-3.79178],'cal/mol/K','+|-',[1.41421,1.41421,1.41421,1.41421,1.41421,1.41421,1.41421])

In [6]:
average_thermo_data(HBI_corrections)

ThermoData(Tdata=([300,400,500,600,800,1000,1500],'K'), Cpdata=([-0.977157,-0.655546,-0.881567,-1.34064,-2.12884,-2.7591,-3.79178],'cal/mol/K'), H298=(81.6781,'kcal/mol'), S298=(-0.103705,'cal/mol/K'), comment="""Radical thermo from dong_pio_liang.py and closed shell thermo from dong_pio_liang.py""")

In [7]:
average_thermo_data(HBI_corrections, weighted=True)

ThermoData(Tdata=([300,400,500,600,800,1000,1500],'K'), Cpdata=([-0.977157,-0.655546,-0.881567,-1.34064,-2.12884,-2.7591,-3.79178],'cal/mol/K','+|-',[0,2.59989e-16,0,0,0,1.03995e-15,1.03995e-15]), H298=(81.6781,'kcal/mol'), S298=(-0.103705,'cal/mol/K','+|-',3.24986e-17), comment="""Radical thermo from dong_pio_liang.py and closed shell thermo from dong_pio_liang.py""")

In [8]:
average_thermo_data(HBI_corrections, weighted=True, bounded=True)

ThermoData(Tdata=([300,400,500,600,800,1000,1500],'K'), Cpdata=([-0.977157,-0.655546,-0.881567,-1.34064,-2.12884,-2.7591,-3.79178],'cal/mol/K','+|-',[1.63299,1.63299,1.63299,1.63299,1.63299,1.63299,1.63299]), H298=(81.6781,'kcal/mol','+|-',1.95959), S298=(-0.103705,'cal/mol/K','+|-',1.63299), comment="""Radical thermo from dong_pio_liang.py and closed shell thermo from dong_pio_liang.py""")